# MC-Test Streamlit App Cloud Deployment Guide


Dieses Notebook führt dich Schritt für Schritt durch den **Deploy** der *Multiple-Choice Test Streamlit App* (`mc_test_app/mc_test_app.py`) auf die **Streamlit Community Cloud**. Die Repository-History ist nun bereinigt; zukünftige Deploys laufen ohne Force-Push direkt auf `main` (Subtree-Strategie).


## 1. Überblick & Architektur


Die App schreibt Antworten der Nutzer in eine CSV (`mc_test_answers.csv`). Auf der Community Cloud ist das Dateisystem **persistierend pro Deployment**, aber bei *Redeploy* oder *History-Reset* kann die Datei zurückgesetzt werden. Für **produktive Speicherung** (Leaderboard etc.) später externe Persistenz (z. B. Google Sheets, Supabase, PostgreSQL, S3) nutzen.


**Komponenten:**
- `mc_test_app.py` – Haupt-App (Streamlit)
- `mc_test_answers.csv` – Log-Datei (append-only)
- `requirements.txt` – Dependencies
- `.streamlit/secrets.toml` (optional) oder Secrets UI – Admin-Key


> Empfehlung: Dependencies klein halten → schnellerer Cold Start & weniger Build-Risiko.


| Vorteil | Bedeutung |
|--------|-----------|
| Vollständig kostenlos | Keine Gebühren für Hosting / Deploy |
| Schnelle Iteration | Änderungen werden automatisch gebaut & deployed |
| Einfaches Secrets-Management | API-Keys sicher hinterlegen (`Secrets`) |
| Stabiler App-Pfad | Pfad zur App bleibt gleich (`mc_test_app/mc_test_app.py`) |
| Geringe Einstiegshürde | Kein eigenes Infrastruktur-Setup nötig |


**Architektur (vereinfacht)**


```
Browser User → Streamlit Cloud Container → mc_test_app.py
```


## 2. Minimal notwendige Dateien


```
README.md
requirements.txt
mc_test_app/mc_test_app.py
mc_test_app/mc_test_answers.csv   # optional – wird sonst automatisch erstellt
deploy.sh
Makefile
```


> Falls `mc_test_answers.csv` NICHT existiert: Die App legt sie beim ersten Log automatisch an.


Optional: `.streamlit/config.toml` (Theme) oder Logo; `.github/workflows/deploy.yml` für manuellen CI-Deploy.


## 3. Dependencies (optimieren)


Schlanke Variante möglich (`requirements.cloud.txt`):


```
streamlit>=1.32
pandas>=2.2
numpy
```


Oder bestehendes `requirements.txt` nutzen (größer = langsamerer Build).


## 4. Admin-Key / Secrets


Definiere einen Admin-Key für die Admin-Ansicht.


Web UI (Secrets):
```toml
MC_TEST_ADMIN_KEY = "Admin"
```


Oder `.streamlit/secrets.toml` (nicht für echte Secrets in Public Repos):


```
.streamlit/
  secrets.toml
```
Inhalt:
```toml
MC_TEST_ADMIN_KEY = "Admin"
```


Fallback im Code optional via `st.secrets.get("MC_TEST_ADMIN_KEY")`.


## 5. Pfad prüfen (lokal)


```bash
streamlit run mc_test_app/mc_test_app.py
```


## 6. Git Remotes & Deploy Workflow


Remotes anzeigen:
```bash
git remote -v
```


GitHub Remote hinzufügen (einmalig):
```bash
git remote add github git@github.com:<user>/<repo>.git
```


### Subtree-Deploy direkt auf main (ohne Force)


Solange `main` ausschließlich durch diesen Prozess aktualisiert wird, bleibt die Subtree-History linear → normaler Push reicht.


Deploy-Zyklus (manuell):


```bash
# 1. Working Tree sauber?
git status
# 2. Subtree Split (temporärer Branch)
git subtree split --prefix mc_test_app -b deploy-mc-test
# 3. Normaler Push auf main (kein --force nötig)
git push github deploy-mc-test:main
# 4. Aufräumen
git branch -D deploy-mc-test
```


### Automatisiert mit Skript `deploy.sh`


```bash
chmod +x deploy.sh
./deploy.sh            # nutzt Standard-Remote 'github'
./deploy.sh origin     # alternativ anderes Remote
```


Skript-Eigenschaften:
- Prüft sauberen Working Tree
- Verifiziert Remote-Synchronität (`main` divergiert nicht)
- Erzeugt Subtree-Split, pusht, räumt temporären Branch auf
- Bricht mit klarer Fehlermeldung bei Verletzungen ab


### Makefile Shortcut


```bash
make deploy          # ruft ./deploy.sh github auf
make deploy-remote REMOTE=origin
```


### GitHub Actions manueller Deploy (workflow_dispatch)


Workflow-Datei: `.github/workflows/deploy.yml` – führt `deploy.sh` in sauberem CI-Umfeld aus. In GitHub UI unter Actions → "Deploy (manual)" triggerbar.


## 7. GitHub Actions (CI) Beispiel


```yaml
name: CI
on:
  push:
    branches: [ main ]
jobs:
  build:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: '3.11'
      - name: Install deps
        run: |
          python -m pip install --upgrade pip
          pip install -r requirements.txt --no-cache-dir
      - name: Basic import
        run: |
          python - <<'PY'
import importlib.util, sys, pathlib
p = pathlib.Path('mc_test_app/mc_test_app.py')
print('Testing import:', p)
spec = importlib.util.spec_from_file_location('mc_test_app', p)
m = importlib.util.module_from_spec(spec)
spec.loader.exec_module(m)
print('OK')
PY
```


## 8. Secrets setzen (Cloud UI)


App Settings → Secrets:
```toml
MC_TEST_ADMIN_KEY = "Admin"
```


## 9. Deployment in Streamlit Cloud


1. New App
2. Repo auswählen
3. Branch: main
4. App path: `mc_test_app/mc_test_app.py`
5. (Optional) Alternative requirements file wählen
6. Deploy


## 10. Monitoring & Logs


Manage app → Logs: Fehler / Build-Zeiten prüfen.


## 11. CSV Persistenz & Alternativen


CSV ist okay für Kurs / kleine Nutzung. Für robustere Nutzung:
- Google Sheets
- Supabase / Postgres
- S3 (Export / Backup)


## 12. Erweiterungen (Ideen)


| Feature | Nutzen | Hinweis |
|---------|-------|---------|
| Erweiterte Leaderboards | Motivation | Mehr Aggregationen |
| Mehrsprachigkeit | Internationalisierung | Mapping Layer |
| Externe DB | Persistente Daten | DB Treiber + Migrationsskript |
| Badge im README | Sichtbarkeit | Automatisch nach Deploy updaten |
| Analytics | Nutzungsmuster | DSGVO beachten |


## 13. Checkliste vor Go-Live


- [ ] Lokaler Start erfolgreich
- [ ] CI grün
- [ ] Secrets gesetzt
- [ ] App-Pfad korrekt
- [ ] Keine sensiblen Daten im Repo
- [ ] README verlinkt Cloud-URL


## 14. Zusammenfassung


Linearer Deploy-Prozess ohne Force: Subtree split → normaler Push auf `main` (oder `deploy.sh` / Makefile Target / manueller Workflow) → Streamlit Cloud baut automatisch.